In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from sklearn import svm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Data

In [ ]:
train = pd.read_csv("location_train.csv")
test = pd.read_csv("location_test.csv")
train.head()

In [ ]:
train.info()

In [ ]:
X = train.drop(["ID", "class"], axis=1)
y = train["class"]

X_test = test.drop(["ID"], axis=1)

In [ ]:
X.head()

In [ ]:
y.head()

# Exploratory data analysis

In [ ]:
X.isnull().sum()

In [ ]:
# Appearence of each class
y.value_counts().sort_index()

# Model selection

In [ ]:
cv = 5
verbose = 1

## KNN

In [ ]:
parameters = {
    "n_neighbors":[1, 5, 10, 20, 30, 40, 50], 
    "weights":["uniform", "distance"],
    "metric":["euclidean", "manhattan", "chebyshev", "minkowski", "wminkowski", "seuclidean", "mahalanobis"]}
knn = GridSearchCV(neighbors.KNeighborsClassifier(), parameters, cv=cv, verbose=verbose)
knn.fit(X, y)

In [ ]:
knn_results = pd.DataFrame(knn.cv_results_)
knn_results= knn_results[["param_n_neighbors", "param_weights", "param_metric", "mean_test_score"]]
knn_results.sort_values(["mean_test_score"], ascending=False).head(10)

## Decision Tree

In [ ]:
parameters = {
    "criterion":["gini", "entropy"],
    "splitter":["best", "random"], 
    "max_depth":["None", 5, 10, 20, 30],
    "max_features":["auto", "sqrt", "log2"]}
dt = GridSearchCV(DecisionTreeClassifier(), parameters, cv=cv, verbose=verbose)
dt.fit(X, y)

In [ ]:
dt_results = pd.DataFrame(dt.cv_results_)
dt_results = dt_results[["param_criterion", "param_splitter", "param_max_depth", "param_max_features", "mean_test_score"]]
dt_results.sort_values(["mean_test_score"], ascending=False).head(10)

## Random Forest

In [ ]:
parameters = {
    "n_estimators":[10, 50, 100, 200, 300, 400, 500], 
    "criterion":["gini", "entropy"], 
    "max_depth":["None", 5, 10, 20],
    "max_features":["auto", "sqrt", "log2"]}
rf = GridSearchCV(RandomForestClassifier(), parameters, cv=cv, verbose=verbose)
rf.fit(X, y)

In [ ]:
rf_results = pd.DataFrame(rf.cv_results_)
rf_results = rf_results[["param_n_estimators", "param_criterion", "param_max_depth", "param_max_features", "mean_test_score"]]
rf_results.sort_values(["mean_test_score"], ascending=False).head(10)

## SVM

In [ ]:
parameters = {
    "C":[1, 5, 10, 20],
    "kernel":["linear", "sigmoid", "rbf", "poly"],     
    "degree":[3, 5],
    "gamma":["scale", "auto"],
    "class_weight":["None", "balanced"]}
svm = GridSearchCV(svm.SVC(), parameters, cv=cv, verbose=verbose)
svm.fit(X, y)

In [ ]:
svm_results = pd.DataFrame(svm.cv_results_)
svm_results = svm_results[["param_C", "param_kernel", "param_degree", "param_gamma", "param_class_weight", "mean_test_score"]]
svm_results.sort_values(["mean_test_score"], ascending=False).head(10)

## Logistic Regression

In [ ]:
parameters = {
    "penalty":["l1", "l2", "elasticnet", "none"], 
    "C":[1, 5, 10, 20],
    "class_weight":["None", "balanced"],
    "solver":["saga"]}
logreg = GridSearchCV(LogisticRegression(), parameters, cv=cv, verbose=verbose)
logreg.fit(X, y)

In [ ]:
logreg_results = pd.DataFrame(logreg.cv_results_)
logreg_results = logreg_results[["param_penalty", "param_C", "param_class_weight", "mean_test_score"]]
logreg_results.sort_values(["mean_test_score"], ascending=False).head(10)

# Results

In [ ]:
print(knn_results.sort_values(["mean_test_score"], ascending=False).head(20).to_markdown(index=False))

In [ ]:
print(dt_results.sort_values(["mean_test_score"], ascending=False).head(20).to_markdown(index=False))

In [ ]:
print(rf_results.sort_values(["mean_test_score"], ascending=False).head(20).to_markdown(index=False))

In [ ]:
print(svm_results.sort_values(["mean_test_score"], ascending=False).head(20).to_markdown(index=False))

In [ ]:
print(logreg_results.sort_values(["mean_test_score"], ascending=False).head(20).to_markdown(index=False))

# Final model

In [ ]:
model = svm.SVC(C=5, kernel="rbf", gamma="scale")
model.fit(X, y)

In [ ]:
predictions = pd.DataFrame(test["ID"])
predictions["class"] = model.predict(X_test)

In [ ]:
predictions.to_csv("submission.csv", index=False)